<a href="https://colab.research.google.com/github/DeuklyoungKo/TensorFlowBasicWithPython/blob/main/Tensorflow/07_modelSaving%26Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[모델 저장과 복원](https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko)

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.8.2


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
24/32 [=====================>........] - ETA: 0s - loss: 1.3452 - accuracy: 0.6198
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.1749 - accuracy: 0.6740 - val_loss: 0.7434 - val_accuracy: 0.7650
Epoch 2/10
31/32 [============================>.] - ETA: 0s - loss: 0.4202 - accuracy: 0.8760
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.4181 - accuracy: 0.8770 - val_loss: 0.5272 - val_accuracy: 0.8330
Epoch 3/10
22/32 [===================>..........] - ETA: 0s - loss: 0.2926 - accuracy: 0.9176
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 8ms/step - loss: 0.2903 - accuracy: 0.9190 - val_loss: 0.5122 - val_accuracy: 0.8510
Epoch 4/10
22/32 [===================>..........] - ETA: 0s - loss: 0.2085 - accuracy: 0.9446
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - los

In [5]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [6]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3186 - accuracy: 0.0970 - 236ms/epoch - 7ms/step
훈련되지 않은 모델의 정확도:  9.70%


In [7]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4140 - accuracy: 0.8700 - 82ms/epoch - 3ms/step
복원된 모델의 정확도: 87.00%


In [8]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [9]:
os.listdir(checkpoint_dir)

['cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'checkpoint',
 'cp-0035.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0015.ckpt.index']

In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4817 - accuracy: 0.8750 - 240ms/epoch - 7ms/step
복원된 모델의 정확도: 87.50%


## 전체 모델 저장하기

In [12]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1240 - accuracy: 0.6910
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4161 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2869 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2042 - accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1566 - accuracy: 0.9660
INFO:tensorflow:Assets written to: saved_model/my_model/assets


### 저장된 모델로부터 새로운 케라스 모델을 로드합니다:

In [15]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4259 - accuracy: 0.8680 - 172ms/epoch - 5ms/step
복원된 모델의 정확도: 86.80%
(1000, 10)


### HDF5 파일로 저장하기

In [17]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1752 - accuracy: 0.6730
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4367 - accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2882 - accuracy: 0.9300
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2207 - accuracy: 0.9420
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1611 - accuracy: 0.9640


In [18]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [19]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4294 - accuracy: 0.8570 - 177ms/epoch - 6ms/step
복원된 모델의 정확도: 85.70%
